In [199]:
import ast
from PIL import Image
from PIL.ExifTags import GPSTAGS
import re
import piexif
import math

Image path input and GSD calculation

In [312]:
image_path = '/home/Praca/Unet/zdjecie/DJI_0642.JPG' # Path to input image
# Calculating ground sampling distance
flight_hight = 40 #meters
s_x = 0.00000241 #meters
focal_length = 0.01026 #meters
gsd = (flight_hight*s_x)/focal_length #meters
gsd = round(gsd,6)

print(gsd)

0.009396


Getting coordinates of input image center

In [201]:
def get_image_metadata(image_path):
    """Extracts image width, height, and GPS coordinates from EXIF metadata."""
    img = Image.open(image_path)
    width, height = img.size  # Get original dimensions
    
    exif_data = piexif.load(img.info.get("exif", b""))  # Load EXIF data

    gps_info = exif_data.get("GPS", {})
    gps_data = {}

    if gps_info:
        for key, val in gps_info.items():
            tag = GPSTAGS.get(key, key)
            gps_data[tag] = val

    # Convert GPS DMS format to Decimal Degrees
    def convert_to_degrees(value):
        """Convert EXIF GPS coordinates to decimal degrees."""
        d = value[0][0] / value[0][1]  # Degrees
        m = value[1][0] / value[1][1]  # Minutes
        s = value[2][0] / value[2][1]  # Seconds
        return d + (m / 60.0) + (s / 3600.0)

    if "GPSLatitude" in gps_data and "GPSLongitude" in gps_data:
        lat = convert_to_degrees(gps_data["GPSLatitude"])
        lon = convert_to_degrees(gps_data["GPSLongitude"])

        if gps_data["GPSLatitudeRef"] == b'S':  # South is negative
            lat = -lat
        if gps_data["GPSLongitudeRef"] == b'W':  # West is negative
            lon = -lon
    else:
        lat, lon = None, None

    return width, height, lat, lon


orig_w, orig_h, lat, lon = get_image_metadata(image_path)
print(f"Original Size: {orig_w}x{orig_h}, GPS: ({lat}, {lon})")


Original Size: 5472x3648, GPS: (54.13496063888889, 19.93641866666667)


Resizing image to fit into model

In [302]:
def resize_image(image_path, output_size=(2048, 1365), save_path="resized_image.jpg"):
    """Resizes the image to 2048x1365 using LANCZOS interpolation and saves it."""
    img = Image.open(image_path)
    img_resized = img.resize(output_size, Image.LANCZOS)  # Use LANCZOS instead of ANTIALIAS
    
    return img_resized

Centroid transformation to ground coordinates

In [203]:
def pixel_to_ground_coordinates(gsd, center_lat, center_lon, old_w, old_h, new_w, new_h, xx, yy):
    """
    Converts pixel coordinates (xx, yy) to geographic coordinates (latitude, longitude).
    """
    # Calculate scale factor
    scale_x = old_w / new_w
    scale_y = old_h / new_h

    # Convert back to original scale
    orig_cx = xx * scale_x
    orig_cy = yy * scale_y

    # Compute pixel offset from the image center
    delta_x = (orig_cx - old_w / 2) * gsd
    delta_y = (orig_cy - old_h / 2) * gsd

    # Convert pixel offsets to degrees
    earth_radius = 6378137.0  # Earth's radius in meters
    lat_per_meter = 1 / (earth_radius * (math.pi / 180))  # Meters to degrees latitude
    lon_per_meter = lat_per_meter / math.cos(math.radians(center_lat))  # Meters to degrees longitude

    # Compute new latitude and longitude
    new_lat = center_lat - delta_y * lat_per_meter  # Negative delta_y moves north
    new_lon = center_lon + delta_x * lon_per_meter  # Positive delta_x moves east

    return new_lat, new_lon

Loading model output data and computing centroids

In [204]:
def extract_data_from_output(data):
    """
    Extracts bounding box coordinates and class IDs dynamically, 
    ensuring correct order even if the CSV format changes.
    """
    if not data or 'output' not in data[0]:
        print("❌ Error: Invalid input data format")
        return None

    # Extract CSV content (skip the header)
    lines = data[0]['output'].strip().split("\n")

    if len(lines) < 2:
        print("❌ Error: CSV data is missing values")
        return None

    header = lines[0].split(",")  # Extract column names
    csv_content = lines[1]  # Extract actual values
    
    # Debug: Print detected header
    print(f"🛠 Detected Header: {header}")

    # Use regex to extract list-like substrings
    matches = re.findall(r'\[.*?\]', csv_content)

    if len(matches) != len(header) - 1:
        print("❌ Error: Mismatch in detected values and header length")
        print(f"🔍 Extracted Matches: {matches}")
        return None

    # Dictionary to store extracted values dynamically
    extracted_values = {}

    # Map extracted values to column names dynamically
    for col_name, match in zip(header, matches):
        extracted_values[col_name.strip()] = ast.literal_eval(match)

    # Extract values safely
    try:
        y_max = extracted_values.get("y_max", [])
        y_min = extracted_values.get("y_min", [])
        x_max = extracted_values.get("x_max", [])
        x_min = extracted_values.get("x_min", [])
        class_ids = extracted_values.get("Class ID", [])

        # Ensure class IDs are integers
        class_ids = [int(cid) for cid in class_ids]

        # Compute centroids
        centroids = [((float(xmax) + float(xmin)) / 2, (float(ymax) + float(ymin)) / 2) 
                     for xmax, xmin, ymax, ymin in zip(x_max, x_min, y_max, y_min)]

        # Debug: Print extracted values
        print(f"✅ Extracted Class IDs: {class_ids}")
        print(f"✅ Computed Centroids: {centroids}")

        return centroids, class_ids

    except Exception as e:
        print(f"❌ Error while processing data: {e}")
        return None

Saving centroids to CSV

In [205]:
import csv

def save_centroids_to_csv(centroids, class_ids, output_csv="centroids.csv"):
    """Saves centroids (lat, lon) and class IDs to a CSV file for QGIS."""
    with open(output_csv, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Latitude", "Longitude", "Class ID"])  # CSV header
        
        for (lat, lon), class_id in zip(centroids, class_ids):
            writer.writerow([lat, lon, class_id])

    print(f"Centroids saved to {output_csv}")

Getting corner coordinates of input image

In [258]:
def get_image_corners(center_lat, center_lon, gsd, img_width, img_height):
    """
    Computes the GPS coordinates of the four corners of an image.
    """

    # Compute pixel shifts from center
    half_w = (img_width / 2) * gsd  # Half width in meters
    half_h = (img_height / 2) * gsd  # Half height in meters

    # Convert meters to degrees
    earth_radius = 6378137.0  # Radius of Earth in meters
    lat_per_meter = 1 / (earth_radius * (math.pi / 180))  # Degrees per meter
    lon_per_meter = lat_per_meter / math.cos(math.radians(center_lat))  # Adjust for latitude

    # Calculate corner coordinates
    top_left = (round(center_lat + half_h * lat_per_meter, 20), round(center_lon - half_w * lon_per_meter, 20))
    top_right = (round(center_lat + half_h * lat_per_meter, 20), round(center_lon + half_w * lon_per_meter, 20))
    bottom_left = (round(center_lat - half_h * lat_per_meter, 20), round(center_lon - half_w * lon_per_meter, 20))
    bottom_right = (round(center_lat - half_h * lat_per_meter, 20), round(center_lon + half_w * lon_per_meter, 20))

    return {
        "top_left": top_left,
        "top_right": top_right,
        "bottom_left": bottom_left,
        "bottom_right": bottom_right
    }

Execution of defined functions

In [313]:
orig_w, orig_h, lat, lon = get_image_metadata(image_path)
print(f"Original Size: {orig_w}x{orig_h}, GPS: ({lat}, {lon})")

Original Size: 5472x3648, GPS: (54.13478558333333, 19.9336355)


In [314]:
resized_image = resize_image(image_path)
#resized_image.show()  # Show resized image

In [315]:
# Connecting to model
from inference_sdk import InferenceHTTPClient

client = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="your Robofow api key" #Can't show mine
)

result = client.run_workflow(
    workspace_name="praca-7cas9",
    workflow_id="custom-workflow-2",
    images={
        "image": resized_image # your input image, might need resizing
    },
    use_cache=True # cache workflow definition for 15 minutes
)

In [316]:
print(result)
data=result

[{'output': 'y_max,x_max,x_min,y_min,Class ID,timestamp\n"[850.0, 637.0, 1169.0, 1099.0, 131.0]","[1076.0, 1203.0, 1965.0, 971.0, 419.0]","[945.0, 1069.0, 1821.0, 944.0, 383.0]","[703.0, 553.0, 1016.0, 1068.0, 100.0]","[\'0\', \'1\', \'1\', \'0\', \'1\']",2025-02-04T17:44:33.998620+00:00\n'}]


In [317]:
centroids, class_ids = extract_data_from_output(data)

🛠 Detected Header: ['y_max', 'x_max', 'x_min', 'y_min', 'Class ID', 'timestamp']
✅ Extracted Class IDs: [0, 1, 1, 0, 1]
✅ Computed Centroids: [(1010.5, 776.5), (1136.0, 595.0), (1893.0, 1092.5), (957.5, 1083.5), (401.0, 115.5)]


In [318]:
real_world_coords = []
for cx, cy in centroids:
    new_lat, new_lon = pixel_to_ground_coordinates(gsd, lat, lon, orig_w, orig_h, 2048, 1365, cx, cy)
    real_world_coords.append((new_lat, new_lon))  # Append the tuple correctly

# Print results
for idx, x in enumerate(real_world_coords):
    print(f"Centroid {idx + 1}: Latitude {x[0]}, Longitude {x[1]}")

Centroid 1: Latitude 54.134764379136406, Longitude 19.933630303478928
Centroid 2: Latitude 54.1348053212826, Longitude 19.93367861187852
Centroid 3: Latitude 54.13469309694249, Longitude 19.933970001985998
Centroid 4: Latitude 54.13469512713156, Longitude 19.933609902322132
Centroid 5: Latitude 54.134913485244574, Longitude 19.93339569017575


In [319]:
save_centroids_to_csv(real_world_coords, class_ids)

Centroids saved to centroids.csv


In [320]:
gps_corners = get_image_corners(lat, lon, gsd, orig_w, orig_h)
print(gps_corners)

{'top_left': (54.1349395393376, 19.93324133425356), 'top_right': (54.1349395393376, 19.934029665746444), 'bottom_left': (54.134631627329064, 19.93324133425356), 'bottom_right': (54.134631627329064, 19.934029665746444)}


In [321]:
# Export to CSV
csv_filename = "image_corners.csv"
with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Corner", "Latitude", "Longitude"])
    for corner, (lat, lon) in gps_corners.items():
        writer.writerow([corner, lat, lon])

print(f"GPS corners saved to {csv_filename}")

GPS corners saved to image_corners3.csv
